## 1. Cài đặt thư viện

In [1]:
!pip install -q llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-vector-stores-faiss
!pip install -q faiss-cpu beautifulsoup4 requests tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, w

## 2. Import libraries và setup

In [2]:
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from typing import List, Dict

# LlamaIndex
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# Setup OpenAI API key từ Colab Secrets
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

print("✅ Setup completed!")

✅ Setup completed!


## 3. Cấu hình LlamaIndex và GRPO Format

In [3]:
# Cấu hình LLM và Embedding
llm = OpenAI(model="gpt-4o-mini", temperature=0.7)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

# Define GRPO reasoning format markers
reasoning_start = "<start_working_out>"
reasoning_end = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

print("✅ LlamaIndex configured with GPT-4o-mini")
print(f"✅ GRPO format markers configured:")
print(f"   Reasoning: {reasoning_start} ... {reasoning_end}")
print(f"   Solution: {solution_start} ... {solution_end}")

✅ LlamaIndex configured with GPT-4o-mini
✅ GRPO format markers configured:
   Reasoning: <start_working_out> ... <end_working_out>
   Solution: <SOLUTION> ... </SOLUTION>


## 4. Crawl văn bản pháp luật từ VBPL (Tái sử dụng function)

In [4]:
def crawl_legal_document(url: str) -> str:
    """
    Crawl nội dung văn bản pháp luật từ VBPL.vn
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm nội dung chính - VBPL thường dùng các selectors này
        content_text = ""

        # Cách 1: Tìm div có id hoặc class chứa "content", "fulltext", "noidung"
        selectors = [
            {'id': 'divContent'},
            {'id': 'ctl00_Content'},
            {'class': 'content1'},
            {'class': 'fulltext'},
            {'class': 'noidung'},
            {'id': 'content'},
        ]

        for selector in selectors:
            content_div = soup.find('div', selector)
            if content_div:
                content_text = content_div.get_text(separator='\n', strip=True)
                if len(content_text) > 500:  # Nội dung đủ dài
                    break

        # Cách 2: Nếu không tìm thấy, tìm trong body và loại bỏ script, style
        if len(content_text) < 500:
            # Xóa các thẻ không cần thiết
            for tag in soup(['script', 'style', 'nav', 'header', 'footer', 'aside']):
                tag.decompose()

            # Lấy text từ body
            body = soup.find('body')
            if body:
                content_text = body.get_text(separator='\n', strip=True)

        # Làm sạch text
        content_text = re.sub(r'\n\s*\n', '\n\n', content_text)
        lines = content_text.split('\n')
        cleaned_lines = [line for line in lines if len(line.strip()) > 10]
        content_text = '\n'.join(cleaned_lines)

        return content_text

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        return ""

# Crawl văn bản (sử dụng cùng URL)
document_url = "https://www.moj.gov.vn/vbpq/lists/vn%20bn%20php%20lut/view_detail.aspx?itemid=23311"
print(f"📥 Crawling document from: {document_url}")
legal_text = crawl_legal_document(document_url)

print(f"\n✅ Crawled {len(legal_text):,} characters")
print(f"📄 Number of lines: {len(legal_text.splitlines())}")
print(f"\nPreview (first 1000 chars):\n{legal_text[:1000]}...")

📥 Crawling document from: https://www.moj.gov.vn/vbpq/lists/vn%20bn%20php%20lut/view_detail.aspx?itemid=23311

✅ Crawled 62,877 characters
📄 Number of lines: 602

Preview (first 1000 chars):
Turn on more accessible mode
Turn off more accessible mode
Cổng thông tin điện tử Bộ Tư pháp - Ministry of Justice’s portal
Cổng thông tin điện tử
CSDLQG về văn bản pháp luật
Chương VIII
Cục Công nghệ thông tin, Bộ Tư pháp trân trọng cảm ơn Quý độc giả trong thời gian qua đã sử dụng hệ thống văn bản quy phạm pháp luật tại địa chỉ
http://www.moj.gov.vn/pages/vbpq.aspx
Đến nay, nhằm phục vụ tốt hơn nhu cầu khai thác, tra cứu văn bản quy phạm pháp luật từ Trung ương đến địa phương, Cục Công nghệ thông tin đã đưa Cơ sở dữ liệu quốc gia về văn bản pháp luật vào sử dụng tại địa chỉ
http://vbpl.vn/Pages/portal.aspx
để thay thế cho hệ thống cũ nói trên.
Cục Công nghệ thông tin trân trọng thông báo tới Quý độc giả được biết và mong rằng Cơ sở dữ liệu quốc gia về văn bản pháp luật sẽ tiếp tục là địa chỉ tin 

## 5. Chunking và tạo Documents (Cùng cách với file gốc)

In [5]:
# Tạo Document từ text crawled
document = Document(text=legal_text)

# Chunking với SentenceSplitter (cùng settings)
splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separator="\n"
)

nodes = splitter.get_nodes_from_documents([document])

print(f"✅ Created {len(nodes)} chunks")
print(f"\nSample chunk:\n{nodes[0].text[:300]}...")

✅ Created 69 chunks

Sample chunk:
Turn on more accessible mode
Turn off more accessible mode
Cổng thông tin điện tử Bộ Tư pháp - Ministry of Justice’s portal
Cổng thông tin điện tử
CSDLQG về văn bản pháp luật
Chương VIII
Cục Công nghệ thông tin, Bộ Tư pháp trân trọng cảm ơn Quý độc giả trong thời gian qua đã sử dụng hệ thống văn bản...


## 6. Tạo FAISS Vector Store và merge similar chunks (Cùng logic)

In [6]:
# Tạo FAISS index
dimension = 1536  # text-embedding-3-small dimension
faiss_index = faiss.IndexFlatL2(dimension)

# Tạo vector store
vector_store = FaissVectorStore(faiss_index=faiss_index)

# Tạo index từ nodes
print("🔄 Creating vector index...")
index = VectorStoreIndex(
    nodes=nodes,
    vector_store=vector_store,
    show_progress=True
)

print(f"✅ Vector index created with {len(nodes)} nodes")

🔄 Creating vector index...


Generating embeddings:   0%|          | 0/69 [00:00<?, ?it/s]

✅ Vector index created with 69 nodes


In [7]:
def merge_similar_chunks(nodes: List, similarity_threshold: float = 0.75) -> List[Dict]:
    """
    Merge các chunks có cosine similarity cao để tạo contexts phong phú hơn
    """
    print(f"🔄 Merging similar chunks (threshold={similarity_threshold})...")
    print(f"📊 Input: {len(nodes)} chunks")

    # Get embeddings cho tất cả nodes
    embeddings = []
    for node in tqdm(nodes, desc="Getting embeddings"):
        emb = embed_model.get_text_embedding(node.text)
        embeddings.append(emb)

    embeddings = np.array(embeddings)

    # Tính cosine similarity matrix
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_matrix = cosine_similarity(embeddings)

    # Merge chunks
    merged_contexts = []
    used_indices = set()

    for i in range(len(nodes)):
        if i in used_indices:
            continue

        # Tìm các chunks similar
        similar_indices = np.where(similarity_matrix[i] > similarity_threshold)[0]
        similar_indices = [idx for idx in similar_indices if idx not in used_indices]

        # Merge text
        merged_text = "\n\n".join([nodes[idx].text for idx in similar_indices])

        merged_contexts.append({
            'context': merged_text,
            'num_chunks': len(similar_indices),
            'chunk_indices': similar_indices
        })

        used_indices.update(similar_indices)

    print(f"✅ Merged {len(nodes)} chunks into {len(merged_contexts)} contexts")
    print(f"📈 Avg chunks per context: {len(nodes)/len(merged_contexts):.1f}")
    return merged_contexts

# Merge chunks với cùng threshold
merged_contexts = merge_similar_chunks(nodes, similarity_threshold=0.75)
print(f"\n📊 Number of merged contexts: {len(merged_contexts)}")

🔄 Merging similar chunks (threshold=0.75)...
📊 Input: 69 chunks


Getting embeddings: 100%|██████████| 69/69 [00:21<00:00,  3.28it/s]

✅ Merged 69 chunks into 43 contexts
📈 Avg chunks per context: 1.6

📊 Number of merged contexts: 43


## 7. Generate Questions từ Contexts (Cùng logic)

In [8]:
def generate_questions_from_context(context: str, num_questions: int = 6) -> List[str]:
    """
    Generate câu hỏi từ context sử dụng GPT-4o-mini
    """
    prompt = f"""Bạn là chuyên gia pháp luật Việt Nam. Dựa trên đoạn văn bản pháp luật dưới đây, hãy tạo {num_questions} câu hỏi hay và đa dạng mà người dùng có thể hỏi.

Văn bản pháp luật:
{context}

Yêu cầu:
- Câu hỏi phải tự nhiên, giống như người dùng thật sẽ hỏi
- Đa dạng về loại câu hỏi: hỏi định nghĩa, điều kiện, thủ tục, quyền lợi, trách nhiệm, v.v.
- Câu hỏi phải có thể trả lời được từ context trên
- Trả về ONLY danh sách các câu hỏi, mỗi câu một dòng, không đánh số

Câu hỏi:"""

    try:
        response = llm.complete(prompt)
        questions_text = response.text.strip()

        # Parse questions
        questions = [q.strip() for q in questions_text.split('\n') if q.strip()]
        # Loại bỏ numbering nếu có
        questions = [re.sub(r'^\d+\.\s*', '', q) for q in questions]

        return questions[:num_questions]
    except Exception as e:
        print(f"❌ Error generating questions: {e}")
        return []

# Test với 1 context
test_context = merged_contexts[0]['context']
test_questions = generate_questions_from_context(test_context, num_questions=6)

print("📝 Sample generated questions:")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

📝 Sample generated questions:
1. Cơ sở dữ liệu quốc gia về văn bản pháp luật là gì?
2. Tôi có thể tìm thấy các văn bản pháp luật nào trên cổng thông tin điện tử của Bộ Tư pháp?
3. Có những điều kiện nào để sử dụng Cơ sở dữ liệu quốc gia về văn bản pháp luật?
4. Thủ tục để tra cứu văn bản quy phạm pháp luật trên cổng thông tin này như thế nào?
5. Quyền lợi của tôi khi sử dụng Cơ sở dữ liệu quốc gia về văn bản pháp luật là gì?
6. Tôi có thể gửi ý kiến góp ý về Cơ sở dữ liệu này ở đâu?


## 8. Generate Structured Answers cho GRPO Format

In [9]:
def generate_structured_answer(question: str, context: str) -> str:
    """
    Generate câu trả lời có cấu trúc GRPO format với reasoning và solution
    """
    prompt = f"""Bạn là trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Văn bản pháp luật:
{context}

Câu hỏi: {question}

Yêu cầu cho phần suy nghĩ (working_out):
- Phân tích câu hỏi: người dùng muốn biết gì?
- Tìm kiếm thông tin liên quan trong văn bản pháp luật
- Xác định điều luật, khoản, điểm áp dụng
- Giải thích logic pháp lý

Yêu cầu cho phần solution:
- Trả lời trực tiếp, rõ ràng
- Trích dẫn điều luật cụ thể
- Độ dài: 30-150 từ
- Dễ hiểu với người dân

Format bắt buộc:
{reasoning_start}
[Phần phân tích và suy nghĩ]
{reasoning_end}

{solution_start}[Câu trả lời chính xác]{solution_end}"""

    try:
        response = llm.complete(prompt)
        answer = response.text.strip()

        # Validate format
        if reasoning_start in answer and reasoning_end in answer and solution_start in answer and solution_end in answer:
            return answer
        else:
            # Fallback: create structure if not properly formatted
            simple_answer = response.text.strip()
            structured = f"""{reasoning_start}
Để trả lời câu hỏi này, tôi cần phân tích văn bản pháp luật được cung cấp và tìm thông tin liên quan đến: {question}

Sau khi xem xét các quy định, tôi sẽ đưa ra câu trả lời chính xác dựa trên pháp luật hiện hành.
{reasoning_end}

{solution_start}{simple_answer}{solution_end}"""
            return structured

    except Exception as e:
        print(f"❌ Error generating structured answer: {e}")
        return ""

# Test với 1 question
test_structured_answer = generate_structured_answer(test_questions[0], test_context)

print("💬 Sample Structured Q&A:")
print(f"Q: {test_questions[0]}")
print(f"A: {test_structured_answer}")
print("\n" + "="*80)

💬 Sample Structured Q&A:
Q: Cơ sở dữ liệu quốc gia về văn bản pháp luật là gì?
A: <start_working_out>
Câu hỏi yêu cầu người dùng tìm hiểu về "Cơ sở dữ liệu quốc gia về văn bản pháp luật". Điều này cho thấy họ muốn biết định nghĩa, chức năng và mục đích của hệ thống này. Trong văn bản đã cung cấp, có nhắc đến "Cơ sở dữ liệu quốc gia về văn bản pháp luật" như một công cụ để khai thác, tra cứu văn bản quy phạm pháp luật từ Trung ương đến địa phương. Bên cạnh đó, văn bản cũng thông báo rằng hệ thống này đã được đưa vào sử dụng để thay thế cho hệ thống cũ, với mục tiêu phục vụ tốt hơn nhu cầu của người dùng. Do đó, có thể suy ra rằng cơ sở dữ liệu này nhằm cung cấp thông tin pháp lý một cách dễ dàng và thuận tiện cho người sử dụng.
<end_working_out>

<SOLUTION>Cơ sở dữ liệu quốc gia về văn bản pháp luật là hệ thống thông tin điện tử nhằm phục vụ việc khai thác, tra cứu các văn bản quy phạm pháp luật từ Trung ương đến địa phương. Hệ thống này giúp người dân và các cơ quan dễ dàng tiếp cận th

## 9. Generate toàn bộ GRPO dataset (Target: 800+ samples)

In [11]:
def generate_grpo_synthetic_dataset(
    contexts: List[Dict],
    target_samples: int = 2150,
    questions_per_context: int = 50
) -> List[Dict]:
    """
    Generate toàn bộ synthetic dataset với GRPO structured format
    """
    dataset = []

    # Tính số contexts cần dùng
    num_contexts_needed = min(len(contexts), (target_samples // questions_per_context) + 1)

    print(f"🚀 Generating {target_samples} GRPO Q&A pairs from {num_contexts_needed} contexts...")

    for ctx_data in tqdm(contexts[:num_contexts_needed], desc="Processing contexts"):
        context = ctx_data['context']

        # Skip contexts quá ngắn
        if len(context.split()) < 50:
            continue

        # Generate questions
        questions = generate_questions_from_context(context, num_questions=questions_per_context)

        # Generate structured answers cho mỗi question
        for question in questions:
            if len(dataset) >= target_samples:
                break

            structured_answer = generate_structured_answer(question, context)

            if structured_answer and reasoning_start in structured_answer:  # Validate structure
                dataset.append({
                    'question': question,
                    'answer': structured_answer,
                    'format': 'grpo_structured'  # Metadata để phân biệt
                })

        if len(dataset) >= target_samples:
            break

    print(f"\n✅ Generated {len(dataset)} GRPO Q&A pairs")
    return dataset

# Generate dataset
grpo_synthetic_data = generate_grpo_synthetic_dataset(
    merged_contexts,
    target_samples=2150,
    questions_per_context=50
)

🚀 Generating 2150 GRPO Q&A pairs from 43 contexts...


Processing contexts: 100%|██████████| 43/43 [3:34:12<00:00, 298.90s/it]


✅ Generated 2121 GRPO Q&A pairs


## 10. Lưu GRPO dataset dưới dạng JSONL

In [12]:
# Lưu ra file JSONL với tên phân biệt
output_path = 'synthetic_legal_qa_grpo_format.jsonl'

with open(output_path, 'w', encoding='utf-8') as f:
    for item in grpo_synthetic_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ Saved {len(grpo_synthetic_data)} GRPO Q&A pairs to {output_path}")

# Download file (nếu chạy trên Colab)
from google.colab import files
files.download(output_path)

✅ Saved 2121 GRPO Q&A pairs to synthetic_legal_qa_grpo_format.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 11. Thống kê & Preview GRPO dataset

In [ ]:
import pandas as pd

# Tạo DataFrame để phân tích
df = pd.DataFrame(grpo_synthetic_data)

# Thống kê độ dài
df['question_len'] = df['question'].apply(lambda x: len(x.split()))
df['answer_len'] = df['answer'].apply(lambda x: len(x.split()))

# Thống kê cấu trúc GRPO
df['has_reasoning'] = df['answer'].apply(lambda x: reasoning_start in x and reasoning_end in x)
df['has_solution'] = df['answer'].apply(lambda x: solution_start in x and solution_end in x)
df['proper_format'] = df['has_reasoning'] & df['has_solution']

print("📊 GRPO DATASET STATISTICS:")
print("=" * 60)
print(f"Total samples: {len(df):,}")
print(f"Properly formatted: {df['proper_format'].sum():,} ({df['proper_format'].mean()*100:.1f}%)")
print(f"\nQuestion length (words):")
print(df['question_len'].describe())
print(f"\nAnswer length (words):")
print(df['answer_len'].describe())

# Preview samples
print("\n" + "=" * 60)
print("SAMPLE GRPO Q&A PAIRS:")
print("=" * 60)
for i in range(min(3, len(df))):
    print(f"\n[Sample {i+1}]")
    print(f"Q: {df.iloc[i]['question']}")
    print(f"A: {df.iloc[i]['answer'][:400]}...")
    print("-" * 60)

## 12. Validation & Quality Check cho GRPO Format

In [ ]:
# Kiểm tra chất lượng format GRPO
print("🔍 GRPO FORMAT QUALITY CHECKS:")
print("=" * 60)

# Check duplicates
duplicate_questions = df[df.duplicated(subset=['question'], keep=False)]
print(f"Duplicate questions: {len(duplicate_questions)}")

# Check format structure
missing_reasoning = df[~df['has_reasoning']]
print(f"Missing reasoning format: {len(missing_reasoning)}")

missing_solution = df[~df['has_solution']]
print(f"Missing solution format: {len(missing_solution)}")

# Check empty sections
def extract_reasoning_section(answer):
    """Extract text between reasoning markers"""
    try:
        start_idx = answer.find(reasoning_start) + len(reasoning_start)
        end_idx = answer.find(reasoning_end)
        return answer[start_idx:end_idx].strip()
    except:
        return ""

def extract_solution_section(answer):
    """Extract text between solution markers"""
    try:
        start_idx = answer.find(solution_start) + len(solution_start)
        end_idx = answer.find(solution_end)
        return answer[start_idx:end_idx].strip()
    except:
        return ""

# Check reasoning section quality
reasoning_lengths = []
solution_lengths = []

for answer in df['answer']:
    reasoning = extract_reasoning_section(answer)
    solution = extract_solution_section(answer)
    reasoning_lengths.append(len(reasoning.split()))
    solution_lengths.append(len(solution.split()))

df['reasoning_len'] = reasoning_lengths
df['solution_len'] = solution_lengths

print(f"\nReasoning section length (words):")
print(f"  Mean: {np.mean(reasoning_lengths):.1f}")
print(f"  Min: {min(reasoning_lengths)}, Max: {max(reasoning_lengths)}")

print(f"\nSolution section length (words):")
print(f"  Mean: {np.mean(solution_lengths):.1f}")
print(f"  Min: {min(solution_lengths)}, Max: {max(solution_lengths)}")

# Check for very short sections
short_reasoning = sum(1 for x in reasoning_lengths if x < 10)
short_solution = sum(1 for x in solution_lengths if x < 5)

print(f"\nVery short reasoning (< 10 words): {short_reasoning}")
print(f"Very short solution (< 5 words): {short_solution}")

print("\n✅ GRPO format quality check completed!")

# Sample format validation
print("\n" + "=" * 60)
print("SAMPLE FORMAT VALIDATION:")
sample_answer = df.iloc[0]['answer']
reasoning_text = extract_reasoning_section(sample_answer)
solution_text = extract_solution_section(sample_answer)

print(f"\nExtracted Reasoning:\n{reasoning_text}")
print(f"\nExtracted Solution:\n{solution_text}")
print(f"\nFormat valid: {bool(reasoning_text and solution_text)}")

## 13. Export cho SFT Training format

In [ ]:
# Tạo format training cho SFT với system prompt
system_prompt = f"""Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Câu trả lời cần dựa trên quy định pháp luật hiện hành và phải rõ ràng, dễ hiểu."""

# Convert sang format training
training_data = []
for item in grpo_synthetic_data:
    if item['answer'] and reasoning_start in item['answer']:  # Only properly formatted ones
        training_item = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": item['question']},
                {"role": "assistant", "content": item['answer']}
            ]
        }
        training_data.append(training_item)

# Lưu training format
training_output_path = 'synthetic_legal_qa_grpo_training.jsonl'

with open(training_output_path, 'w', encoding='utf-8') as f:
    for item in training_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ Saved {len(training_data)} training examples to {training_output_path}")
print(f"📈 Ready for SFT training with GRPO format!")

# Download training file
files.download(training_output_path)

print("\n🎯 SUMMARY:")
print(f"- Generated {len(grpo_synthetic_data)} synthetic samples")
print(f"- {len(training_data)} properly formatted for GRPO training")
print(f"- Format includes {reasoning_start}...{reasoning_end} and {solution_start}...{solution_end}")
print(f"- Ready for SFT training to align with your GRPO model output format")